In [6]:
# 원본은 내 pc(Dell)의 deepcell-speechbrain-2 Docker에 있음

# import speechbrain
# from speechbrain.utils.edit_distance import wer_details_for_batch
# import pandas as pd
import numpy as np
from typing import List

In [1]:
import sys
sys.path

['/workspace/speechbrain/recipes/KdialectSpeech/ASR/sWER',
 '/opt/conda/lib/python38.zip',
 '/opt/conda/lib/python3.8',
 '/opt/conda/lib/python3.8/lib-dynload',
 '',
 '/opt/conda/lib/python3.8/site-packages',
 '/workspace/speechbrain',
 '/opt/conda/lib/python3.8/site-packages/IPython/extensions',
 '/root/.ipython']

In [7]:
# 음절 단위 토큰화, 빈칸을 _로 표시

def char_tokenizer(s):
    result = []
    flag = False
    for c in s:
        if c == ' ':
            flag = True
            continue

        if flag == True:
            c = '_' + c
            flag = False

        result.append(c)
        
    return result

print(char_tokenizer('나는 어제 치킨을 먹었다'))

['나', '는', '_어', '제', '_치', '킨', '을', '_먹', '었', '다']


In [3]:
def remove_space_symbol(s):
    return s.replace(' ', '')
    
# print(remove_space_symbol(ref))

In [16]:
def get_norm_text(ref , hyp):
    refs = char_tokenizer(ref)
    hyps = char_tokenizer(hyp)

    ref_nospace = ref.replace(' ', '')
    hyp_nospace = hyp.replace(' ', '')
    print(f'ref_nospace : {ref_nospace}')
    print(f'hyp_nospace : {hyp_nospace}')

    rlen = len(refs)
    hlen = len(hyps)
    print(f'rlen : {rlen} , hlen : {hlen}')

    scores =  np.zeros((hlen+1, rlen+1), dtype=np.int32)

    # initialize, 공란을 무시하고 음절의 거리 매트릭스 만들기
    for r in range(rlen+1):
        scores[0, r] = r
        
    for h in range(1, hlen+1):
        scores[h, 0] = scores[h-1, 0] + 1
        for r in range(1, rlen+1):
            sub_or_cor = scores[h-1, r-1] + (0 if ref_nospace[r-1] == hyp_nospace[h-1] else 1)
            insert = scores[h-1, r] + 1
            delete = scores[h, r-1] + 1

            scores[h, r] = min(sub_or_cor, insert, delete)
    print(f'lev scores--------------------------')
    print(scores)

    # traceback and compute alignment
    h, r = hlen, rlen
    ref_norm, hyp_norm = [], []

    while r > 0 or h > 0:
        print('while start *****')
        print(f'ref_nospace[r-1] : {ref_nospace[r-1]}')
        print(f'hyp_nospace[h-1] : {hyp_nospace[h-1]}')
        if h == 0:
            last_r = r - 1
        elif r == 0:
            last_h = h - 1
            last_r = r
        else:
            sub_or_cor = scores[h-1, r-1] + (0 if ref_nospace[r-1] == hyp_nospace[h-1] else 1)
            insert = scores[h-1, r] + 1
            delete = scores[h, r-1] + 1

            if sub_or_cor < min(insert, delete):
                last_h, last_r = h - 1, r - 1
            else:
                last_h, last_r = (h-1, r) if insert < delete else (h, r-1)

            print(f'last_h : {last_h}')
            print(f'last_r : {last_r}')

            c_hyp = hyps[last_h] if last_h == h-1 else ''
            c_ref = refs[last_r] if last_r == r-1 else ''
            print(f'c_hyp : {c_hyp}')
            print(f'c_ref : {c_ref}')

            h, r = last_h, last_r

            # do word-spacing normalization
            if c_hyp.replace('_', '') == c_ref.replace('_', ''):
                c_hyp = c_ref

        ref_norm.append(c_ref)
        hyp_norm.append(c_hyp)
        print(f'ref_norm : {ref_norm}')
        print(f'hyp_norm : {hyp_norm}')
    
    return ref_norm[::-1], hyp_norm[::-1]

In [4]:
ref = '나는 어제 양념 치킨' # 양념 이란 단어가 ref에만 있으므로 처리에 문제가 있다. 
hyp = '나는어제  치킨'

# ref = '나는 어제 양념 치킨을 먹었다' # 양념 이란 단어가 ref에만 있으므로 처리에 문제가 있다. 
# hyp = '나는어제  치킨을먹었다'

# ref = '음성인식 평가 방법을 소개합니다'
# hyp = '어 음성 인식 평가방버블 소개합니다'
# -> 어 음성인식 평가 방버블 소개합니다

# ref = '음성인식 방법을 소개합니다'
# hyp = '음성 인식 방버블 소개합니다'

In [17]:
ref_n, hyp_n = get_norm_text(ref, hyp)

ref_nospace : 나는어제양념치킨
hyp_nospace : 나는어제치킨
rlen : 8 , hlen : 6
lev scores--------------------------
[[0 1 2 3 4 5 6 7 8]
 [1 0 1 2 3 4 5 6 7]
 [2 1 0 1 2 3 4 5 6]
 [3 2 1 0 1 2 3 4 5]
 [4 3 2 1 0 1 2 3 4]
 [5 4 3 2 1 1 2 2 3]
 [6 5 4 3 2 2 2 3 2]]
while start *****
ref_nospace[r-1] : 킨
hyp_nospace[h-1] : 킨
last_h : 5
last_r : 7
c_hyp : 킨
c_ref : 킨
ref_norm : ['킨']
hyp_norm : ['킨']
while start *****
ref_nospace[r-1] : 치
hyp_nospace[h-1] : 치
last_h : 4
last_r : 6
c_hyp : _치
c_ref : _치
ref_norm : ['킨', '_치']
hyp_norm : ['킨', '_치']
while start *****
ref_nospace[r-1] : 념
hyp_nospace[h-1] : 제
last_h : 4
last_r : 5
c_hyp : 
c_ref : 념
ref_norm : ['킨', '_치', '념']
hyp_norm : ['킨', '_치', '']
while start *****
ref_nospace[r-1] : 양
hyp_nospace[h-1] : 제
last_h : 4
last_r : 4
c_hyp : 
c_ref : _양
ref_norm : ['킨', '_치', '념', '_양']
hyp_norm : ['킨', '_치', '', '']
while start *****
ref_nospace[r-1] : 제
hyp_nospace[h-1] : 제
last_h : 3
last_r : 3
c_hyp : 제
c_ref : 제
ref_norm : ['킨', '_치', '념', '_양', '제']
h

In [18]:
hyp_n

['나', '는', '_어', '제', '', '', '_치', '킨']

In [19]:
ref_n

['나', '는', '_어', '제', '_양', '념', '_치', '킨']

In [21]:
result = ''.join(map(str, hyp_n))
result

'나는_어제_치킨'

In [8]:
# map(function, iterable)
# '구분자'.join(리스트)

result = ''.join(map(str, ref_n)).replace('_', ' ')
print(result)

나는 어제 양념 치킨을 먹었다


In [9]:
result = ''.join(map(str, hyp_n)).replace('_', ' ')
print(result)

나는 어제 치킨을 먹었다


In [22]:
def get_swords(ref , hyp):
    refs = char_tokenizer(ref)
    hyps = char_tokenizer(hyp)
    ref_nospace = ref.replace(' ', '')
    hyp_nospace = hyp.replace(' ', '')
    rlen = len(refs)
    hlen = len(hyps)
    scores =  np.zeros((hlen+1, rlen+1), dtype=np.int32)

    # initialize, 공란을 무시하고 음절의 거리 매트릭스 만들기
    for r in range(rlen+1):
        scores[0, r] = r
    for h in range(1, hlen+1):
        scores[h, 0] = scores[h-1, 0] + 1
        for r in range(1, rlen+1):
            sub_or_cor = scores[h-1, r-1] + (0 if ref_nospace[r-1] == hyp_nospace[h-1] else 1)
            insert = scores[h-1, r] + 1
            delete = scores[h, r-1] + 1
            scores[h, r] = min(sub_or_cor, insert, delete)

    # traceback and compute alignment
    h, r = hlen, rlen
    ref_norm, hyp_norm = [], []

    while r > 0 or h > 0:
        if h == 0:
            last_r = r - 1
        elif r == 0:
            last_h = h - 1
            last_r = r
        else:
            sub_or_cor = scores[h-1, r-1] + (0 if ref_nospace[r-1] == hyp_nospace[h-1] else 1)
            insert = scores[h-1, r] + 1
            delete = scores[h, r-1] + 1

            if sub_or_cor < min(insert, delete):
                last_h, last_r = h - 1, r - 1
            else:
                last_h, last_r = (h-1, r) if insert < delete else (h, r-1)

            c_hyp = hyps[last_h] if last_h == h-1 else ''
            c_ref = refs[last_r] if last_r == r-1 else ''
            h, r = last_h, last_r

            # do word-spacing normalization
            if c_hyp.replace('_', '') == c_ref.replace('_', ''):
                c_hyp = c_ref

        ref_norm.append(c_ref)
        hyp_norm.append(c_hyp)

    # ref_norm[::-1], hyp_norm[::-1]
    shyp = ''.join(map(str, hyp_norm[::-1])).replace('_', ' ')
    return shyp

In [28]:
ref = '나는 어제 양념치킨을 먹었다' # 양념 이란 단어가 ref에만 있으므로 처리에 문제가 있다. 
hyp = '나는어제  치킨을먹었다'

# hyp = '나는 어제 양념 치킨을 먹었다' # 양념 이란 단어가 ref에만 있으므로 처리에 문제가 있다. 
# ref = '나는어제  치킨을먹었다'

In [29]:
get_swords(ref, hyp)

'나는 어제치킨을 먹었다'

In [24]:
get_swords(ref, hyp)

'나는 어제 치킨을 먹었다'

In [42]:
hyp.split()

['나는어제', '치킨을먹었다']

In [43]:
ref_list = ref.split()
hyp_list = hyp.split()

In [44]:
hyp_list

['나는어제', '치킨을먹었다']

In [10]:
# 음절 단위 토큰화, 빈칸을 _로 표시

def list_tokenizer(l):
    '''
    단어들의 리스트를 문자들의 리스트로 만든다. 이 때 단어의 맨 앞의 문자에는 밑줄(_)을 붙인다.

    Arguments
    ---------
    l : list
        단어(문자열)들의 리스트

    Retrun
    ---------
    result : list
        단어(문자열)들의 리스트
    '''
    result = []
    flag = False
    for word in l:
        result.append('_' + word[0])
        result.extend(list(word[1:]))

    result[0] = result[0].replace('_', '')
    return result

target_word = ['잔칫날', '아칙에', '신랑이', '새각시', '돌레', '오민', '새각신', '고부랑건지', '허곡', '가메', '탕', '신랑', '집이', '갓어']
print(list_tokenizer(target_word))

['잔', '칫', '날', '_아', '칙', '에', '_신', '랑', '이', '_새', '각', '시', '_돌', '레', '_오', '민', '_새', '각', '신', '_고', '부', '랑', '건', '지', '_허', '곡', '_가', '메', '_탕', '_신', '랑', '_집', '이', '_갓', '어']


In [37]:
def space_normalize_lists(ref_list, hyp_list):
    '''
    두 개의 단어들의 리스트를 비교하여 띄어쓰기를 맞추어 준다.(space normalize)
    ref_list를 기준으로 hyp_list의 띄어쓰기를 맞춘다.

    Arguments
    ---------
    ref_list : list
        단어(문자열)들의 리스트
        이 리스트를 기준으로 hyp_list의 띄어쓰기를 맞춘다.
    hyp_list : list
        단어(문자열)들의 리스트

    Retrun
    ---------
    result : list
        ref_list에 띄어쓰기를 맞춘 hyp_list의 리스트
    '''

    ref_nospace = ''.join(ref_list)
    hyp_nospace = ''.join(hyp_list)
    # print(f'ref_nospace : {ref_nospace}')
    # print(f'hyp_nospace : {hyp_nospace}')

    refs = list_tokenizer(ref_list)
    hyps = list_tokenizer(hyp_list)
    # print(f'refs : {refs}')
    # print(f'hyps : {hyps}')

    rlen = len(refs)
    hlen = len(hyps)
    scores =  np.zeros((hlen+1, rlen+1), dtype=np.int32)


    # initialize, 공란을 무시하고 음절의 거리 매트릭스 만들기
    for r in range(rlen+1):
        scores[0, r] = r
    for h in range(1, hlen+1):
        scores[h, 0] = scores[h-1, 0] + 1
        for r in range(1, rlen+1):
            sub_or_cor = scores[h-1, r-1] + (0 if ref_nospace[r-1] == hyp_nospace[h-1] else 1)
            insert = scores[h-1, r] + 1
            delete = scores[h, r-1] + 1
            scores[h, r] = min(sub_or_cor, insert, delete)
    
    # print(f'lev scores--------------------------')
    # print(scores)

    
    # traceback and compute alignment
    h, r = hlen, rlen
    ref_norm, hyp_norm = [], []

    while r > 0 or h > 0:
        if h == 0:
            last_r = r - 1
        elif r == 0:
            last_h = h - 1
            last_r = r
        else:
            sub_or_cor = scores[h-1, r-1] + (0 if ref_nospace[r-1] == hyp_nospace[h-1] else 1)
            insert = scores[h-1, r] + 1
            delete = scores[h, r-1] + 1

            if sub_or_cor < min(insert, delete):
                last_h, last_r = h - 1, r - 1
            else:
                last_h, last_r = (h-1, r) if insert < delete else (h, r-1)

            c_hyp = hyps[last_h] if last_h == h-1 else ''
            c_ref = refs[last_r] if last_r == r-1 else ''
            h, r = last_h, last_r

            # do word-spacing normalization
            if c_hyp.replace('_', '') == c_ref.replace('_', ''):
                c_hyp = c_ref

        ref_norm.append(c_ref)
        hyp_norm.append(c_hyp)

    while '' in ref_norm:
        ref_norm.remove('')
    while '' in hyp_norm:
        hyp_norm.remove('')
    # print(f'norm--------------------------')
    # print(f'ref_norm : {ref_norm[::-1]}')
    # print(f'hyp_norm : {hyp_norm[::-1]}')

    return hyp_norm[::-1]

    

In [35]:
ref = '나는 어제 양념치킨을 먹었다' # 양념 이란 단어가 ref에만 있으므로 처리에 문제가 있다. 
hyp = '나는어제  치킨을먹었다'

ref_list = ref.split()
hyp_list = hyp.split()

print(ref_list)
print(hyp_list)

['나는', '어제', '양념치킨을', '먹었다']
['나는어제', '치킨을먹었다']


In [38]:
space_normalize_lists(ref_list, hyp_list)

['나', '는', '_어', '제', '치', '킨', '을', '_먹', '었', '다']

In [ ]:
file_name = 'hyp_label.csv'
hyp_label_df = pd.read_csv(file_name)

In [ ]:
_, hyp_label_df['hyp_n'] = get_norm_text(hyp_label_df['wrd'], hyp_label_df['hyp'])

In [ ]:
hyp_label_df

In [ ]:
pd.options.display.max_rows = 100

In [ ]:
print(hyp_label_df['hyp_n'])

In [ ]:
hyp_label_df['hyp_s'] = ''.join(map(str, hyp_label_df['hyp_n'])).replace('_', ' ')

In [ ]:
(hyp_label_df['hyp_n']).replace('_', ' ')

In [ ]:
wer_test = wer_details_for_batch(hyp_label_df['spk_id'], hyp_label_df['wrd'], hyp_label_df['hyp'])
wer_list = [i['WER'] for i in wer_test]

In [ ]:
hyp_label_df['wer'] = wer_list

In [ ]:
hyp_label_df

In [ ]:
# sWER 구현하기

In [31]:
predicted_words = [['그', '이제', '그', '이제', '그', '이제', '그', '이제', '그', '이제', '그', '이제', '그', '이제', '그', '이제', '그', '이제', '그', '이제', '그', '이제', '그', '이제', '그', '이제', '그', '이제', '이제', '그', '이제', '그', '이제', '그', '이제', '그', '이제', '그', '이제', '그', '이제', '그', '이제', '그', '이제', '그', '이제', '그']]
batch_wrd = ['잔칫날 아칙에 신랑이 새각시 돌레 오민 새각신 고부랑건지 허곡 가메 탕 신랑 집이 갓어']
target_words = [['잔칫날', '아칙에', '신랑이', '새각시', '돌레', '오민', '새각신', '고부랑건지', '허곡', '가메', '탕', '신랑', '집이', '갓어']]

In [33]:
predicted_word = predicted_words[0]
underbar_prepended = ['_' + s for s in predicted_word]

In [35]:
underbar_prepended[2]

'_그'

In [46]:
# shyp = ''.join(map(str, hyp_norm[::-1])).replace('_', ' ')
''.join(map(str, target_words[0]))

'잔칫날아칙에신랑이새각시돌레오민새각신고부랑건지허곡가메탕신랑집이갓어'

In [41]:
a = [1.2, 2.5, 3.7, 4.6]
# a = list(map(int, a))
a
for i, v in enumerate(a):
    print(f'{i}, {v}')


0, 1.2
1, 2.5
2, 3.7
3, 4.6


In [42]:
a[1]

2.5

In [48]:
map(int, a)

In [49]:
''.join(target_words[0])

'잔칫날아칙에신랑이새각시돌레오민새각신고부랑건지허곡가메탕신랑집이갓어'